In [32]:
using Physical

function ndgrid{T}(v1::AbstractVector{T}, v2::AbstractVector{T})
    m, n = length(v1), length(v2)
    v1 = reshape(v1, m, 1)
    v2 = reshape(v2, 1, n)
    (repmat(v1, 1, n), repmat(v2, m, 1))
end

function BK_simulator(phi::Vector; model_id=0, Ca=([0, 0.7, 4, 12, 22, 55, 70, 95]*1e-6), V=collect(-100:25:200)*1e-3)
    # Simulates the MWC model (Aldrich 2002) open probabilities of BK given input parameters Phi over the given Ca-V grid points
    # model_id=0 is the full model, further models are reduced representations by MBAM
    Temp = 304 * Kelvin
    elec_resp = (Volt*(-1)*e_electron/(k_boltzmann*304*Kelvin));
    
    tmp = ndgrid(Ca, V)
    Ca = tmp[1]
    V = tmp[2]
    
    if model_id==0
        #phi = {L0, zL, J0, Kd, C, D, E}
        L = phi[1]*exp(-phi[2]*V* elec_resp )
        J = phi[3]*exp(-phi[2]*V* elec_resp )
        K = Ca / (phi[4]);
        C = phi[5]
        D = phi[6]
        E = phi[7]
        Popen = ( L .* (1 + K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( L .* (1+ K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K + J.*K.*E).^4);     
    end
    
    if model_id==1
        #phi = {zL, J0, zJ, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        elec_L = exp(-phi[1]*V* elec_resp );
        J = phi[2]*exp(-phi[3]*V* elec_resp )
        K = Ca / (phi[4]);
        C = phi[5]
        D = phi[6]
        E = phi[7]
        
        Popen = ( elec_L .* (K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( elec_L .* (K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K).^4);     
    end
    
    if model_id==2
        #zJ -> 0
        #phi = {zL, J0, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        elec_L = exp(-phi[1]*V* elec_resp );
        J = phi[2]*exp(0);
        K = Ca / (phi[3]);
        C = phi[4]
        D = phi[5]
        E = phi[6]
        
        Popen = ( elec_L .* (K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( elec_L .* (K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K).^4);     
    end
    
    if model_id==3
        #zJ -> 0
        #phi = {zL, J0, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        elec_L = exp(-phi[1]*V* elec_resp );
        J = phi[2]*exp(0);
        K = Ca / (phi[3]);
        C = phi[4]
        D = phi[5]
        E = 0
        
        Popen = ( elec_L .* (K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( elec_L .* (K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K).^4);     
    end
    
    return Popen
end

    

BK_simulator (generic function with 4 methods)

In [33]:

function BK_reduce(phi::Vector; model_id=0, in_log=1)
    if in_log==1
        f_add = (a,x,b,y) -> a*x + b*y
    else
        f_add = (a,x,b,y) -> (x.^a)*(y.^b)
    end

    if model_id==0
        phi_red = phi;
        
        
    elseif model_id==1
        #phi = {zL, J0, zJ, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = phi[2]
        phi_red[2] = phi[3]
        phi_red[3] = phi[4]
        phi_red[4] = f_add(1/4,phi[1], 1, phi[5])
        phi_red[5] = f_add(1/4,phi[1], 1, phi[6])
        phi_red[6] = f_add(1,phi[7], -1/4, phi[1])
        
        
    elseif model_id==2 # Reduce from model 1 to model 2 
        # zJ -> 0
        #phi = {zL, J0, zJ, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = phi[4]
        phi_red[4] = phi[5]
        phi_red[5] = phi[6]
        phi_red[6] = phi[7]
        
     elseif model_id==3 # Reduce from model 2 to model 3 
        # E -> 0
        #phi = {zL, J0, zJ, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1:5] = phi[1:5]
        
    else
        
        phi_red = phi;
    end
    
    
    
    return phi_red;
end

BK_reduce (generic function with 1 method)

In [34]:
function BK_cost(phi; model_id=0, phi_0=[2.2e-6, 0.58, 0.1026, 39*1e-6, 6.16,30.4,2])  #as in Miranda 2010, Fig 4)
    cost = sum((BK_simulator(phi, model_id=model_id) - BK_simulator(phi_0, model_id=0)).^2)
    return cost
end

BK_cost (generic function with 2 methods)